# Predicción de NBV 

En este notebook implementaremos un sistema para reconstrucción via NBV haciendo uso de la arquitectura autoencoder y MLP

* pedirá la dirección a la carpeta contenedora del objeto
* Creará un link simbolico para acceder a las texturas (verifica si no existe ya) y carpetas para almacenar la información obtenida durante el proceso: 
    - Nubes de puntos
    - RGB
    - Profundidad
* Despliega el objeto para verificar si es el que espera el usuario y debe confirmar si es correcto
* Genera una posición y orientación random (podría extraerla de Hintertoiser), importante conocer el bounding box para no colisionar con el objeto
* Inicia el proceso de reconstrucción
* Captura información y almacena 
* Crea grid de 31x31x31
* Procesamiento de IA
* Condición si la cumple repite o finaliza
* Reporte de métricas

In [2]:
import os
from symlink import symbolic_dir
import open3d as o3d
import octomap
import torch
from utils_o3d import Get_Pointcloud, Get_RGBD, Get_octree
from MLP import MLP
from utils import net_sample_output_nbv
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
print("Ingresa la dirección a la carpeta contenedora del banco de datos")
direccion = input("Cúal es la dirección a la carpeta contenedora?:") # Dataset acces
print(list(filter(os.path.isdir, os.listdir(direccion))))
carpeta = input("A que carpeta quieres acceder?: ") #object folder
dir_carpeta = direccion + carpeta
if os.path.lexists(dir_carpeta + "/meshes/texture.png") == False:
    symbolic_dir(dir_carpeta)
    RGB = "/RGB"
    Depth = "/Depth"
    Point_cloud = "/Point_cloud"
    os.mkdir(dir_carpeta + RGB)
    os.mkdir(dir_carpeta + Depth)
    os.mkdir(dir_carpeta + Point_cloud)
    #print("Cree link simbolico")
visualizar = input("Quieres visualizar el objeto? (presiona S en caso de sí):") 
if visualizar == 'S'  or  visualizar == 's' :
    mesh = o3d.io.read_triangle_mesh(dir_carpeta + '/meshes/model.obj', True) 
    o3d.visualization.draw_geometries([mesh])

global p_c

fov = 17.7064
img_H = 100
img_W = 100
up = [0, 1, 0]
cent = [0.5,0.5,0.5]
eye = [2.0,1.0,2.0]
resolution = 0.01

## TODO: Aqui generamos la vista random

print("Inicia el proceso de reconstrucción ...")

condicion = False
i = 0
#Cargamos los modelos de predicción de posición
model= MLP().cuda() 
path_weights = '/position/weights_entrenamiento_MLP_11.pth' ## Modificar direccion de pesos
model.load_state_dict(torch.load(path_weights))
device = torch.cuda.current_device()

#Inicializamos el octomap
resolution = 0.01 # resolucion del octree
octree = octomap.OcTree(resolution) # inicializamos el octree

while condicion == False:
    #Cargamos malla
    mesh = o3d.io.read_triangle_mesh('objetos/' + 'meshes/model.obj', True)

    # Raycasting
    mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(mesh1)
    # render for RGBD images
    
    render = o3d.visualization.rendering.OffscreenRenderer(width=img_W, height=img_H) #Linux only
    render.scene.add_geometry('mesh', mesh)

    # RGBD and pointcloud extraction
    Get_Pointcloud(scene, fov, cent, eye, up, img_W, img_H, dir_carpeta, i)
    Get_RGBD(render,  fov, cent, eye, up, dir_carpeta, i)
    #Occupancy grid
    octree =  Get_octree(pc, i)
    torch_grid = torch.from_numpy(grid)
    
    #IA
    #output, _= net_sample_output_nbv(model, grid, device) 
    
    #Evaluacion
    # Esta completa la reconstrucción?
    # si condicion = False
    # continua
    # si condicion = True 
    # break 
    # Metricas 


Ingresa la dirección a la carpeta contenedora del banco de datos


[]
Inicia el proceso de reconstrucción ...


FileNotFoundError: [Errno 2] No such file or directory: '/position/weights_entrenamiento_MLP_11.pth'

In [ ]:
# TODO: verificar que pasa con la geometria, si es necesario mejor leer la nube de puntos para no tener inconvenientes
#  

In [6]:
x = pc.get_max_bound()
x

array([0.02635956, 0.02606279, 0.08874011])

In [29]:
direccion

'D:/Saulo/ProyectoPHD/MODELDATASET/'

In [34]:
#!/usr/bin/env python

import imgviz
import numpy as np
import trimesh
import trimesh.viewer

import octomap
import sys
sys.path.insert(0, '/home/skunkllr/octomappython/examples/')
from insertPointCloud import pointcloud_from_depth
from insertPointCloud import visualize


data = imgviz.data.arc2017() # Carga dataset
camera_info = data["camera_info"]
K = np.array(camera_info["K"]).reshape(3, 3)
rgb = data["rgb"] #selecciona el dataset a utilizar
pcd = pointcloud_from_depth(
    data["depth"], fx=K[0, 0], fy=K[1, 1], cx=K[0, 2], cy=K[1, 2]
) # Se infiere la profundidad a partir de una imagen RGB

nonnan = ~np.isnan(pcd).any(axis=2) #filtro para valores nan
mask = np.less(pcd[:, :, 2], 2) #mascara    ##### ver que es lo que hace

resolution = 0.01 # resolucion del octree
octree = octomap.OcTree(resolution) # inicializamos el octree
octree.insertPointCloud(
    pointcloud=pcd[nonnan],
    origin=np.array([0, 0, 0], dtype=float),
    maxrange=1.25,
) # se agrega el pointcloud y el origen del sensor

# Se extrae el minimo y maximo de BBox
aabb_min = octree.getMetricMin() 
aabb_max = octree.getMetricMax()
center = (aabb_min + aabb_max) / 2
dimension = np.array([31, 31, 31]) # dimension de la voxelizacion
origin = center - dimension / 2 * resolution

#nuevos BBox  con la nueva resolución
aabb_min = origin - resolution / 2
aabb_max = origin + dimension * resolution + resolution / 2

grid = np.full(dimension, -1, np.int32)
transform = trimesh.transformations.scale_and_translate(
    scale=resolution, translate=origin
) #transformacion
points = trimesh.voxel.VoxelGrid(encoding=grid, transform=transform).points # Voxel grid con los puntos de la nube
labels = octree.getLabels(points) # extrae las etiquetas o valor ocupacional


occupied = points[labels == 1] #Ocupado
empty = points[labels == 0] # Vacio
 # TENEMOS valores en -1 a 1, entonces no se que pasa con el resto de valores (los negativos)
visualize(
    occupied=occupied,
    empty=empty,
    K=K,
    width=camera_info["width"],
    height=camera_info["height"],
    rgb=rgb,
    pcd=pcd,
    mask=mask,
    resolution=resolution,
    aabb=(aabb_min, aabb_max),
)

In [47]:
visualize(
    occupied=occupied,
    empty=empty,
    K=K,
    width=camera_info["width"],
    height=camera_info["height"],
    rgb=rgb,
    pcd=pcd,
    mask=mask,
    resolution=resolution,
    aabb=(aabb_min, aabb_max),
)

In [3]:
pcd.shape

(480, 640, 3)

In [35]:
### Try to translate to Open3d

#Llegamos con las nubes de puntos en un array con dimensiones [:,:,x3]
#impotante retirar punto nan (los vi presentes en una nube de puntos)

resolution = 0.01 # resolucion del octree
octree = octomap.OcTree(resolution) # inicializamos el octree # inicializar mas arriba
octree.insertPointCloud(
    pointcloud=pcd[nonnan],                         # Filtrar valores nan
    origin=np.array([0, 0, 0], dtype=float),
    maxrange=1.25,
) # se agrega el pointcloud y el origen del sensor

# Se extrae el minimo y maximo de BBox
aabb_min = octree.getMetricMin() 
aabb_max = octree.getMetricMax()
center = (aabb_min + aabb_max) / 2
dimension = np.array([31, 31, 31]) # dimension de la voxelizacion
origin = center - dimension / 2 * resolution

#nuevos BBox  con la nueva resolución
aabb_min = origin - resolution / 2
aabb_max = origin + dimension * resolution + resolution / 2

grid = np.full(dimension, -1, np.int32)
transform = trimesh.transformations.scale_and_translate(
    scale=resolution, translate=origin
) #transformacion
points = trimesh.voxel.VoxelGrid(encoding=grid, transform=transform).points # Voxel grid con los puntos de la nube

labels = octree.getLabels(points) # extrae las etiquetas o valor ocupacional
# according to octomap-python # -1: unknown, 0: empty, 1: occupied

#according to our dataset # 0.5: unknown, 0: empty, 1: occupied
labels = labels.astype(dtype = np.float32)
np.place(labels, labels <0, [0.5]) # los valores menores a 0 pasan a ser iguales a 0.5
grid = np.reshape(labels,(31,31,31))

In [146]:
points

TrackedArray([[-0.17 , -0.2  ,  0.455],
              [-0.17 , -0.2  ,  0.465],
              [-0.17 , -0.2  ,  0.475],
              ...,
              [ 0.13 ,  0.1  ,  0.735],
              [ 0.13 ,  0.1  ,  0.745],
              [ 0.13 ,  0.1  ,  0.755]])

### TODO:

* Parece que el error para extraer el octree esta en octomap/octomap/octomap.pyx 
    hay que explorarlo

In [75]:
octree.extractPointCloud()

AttributeError: 'octomap.leaf_iterator' object has no attribute '_leaf_iterator__is_end'